In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class individual:
    
    def __init__(self, initial_range, individual_size):
        self.genes = np.random.uniform(low=-initial_range,high=initial_range,size=(individual_size))
        self.evaluated = False
        self.fitness = np.zeros((8))
        self.player_energy = np.zeros((8))
        self.enemy_energy = np.zeros((8))
        
class log_book():
    
    def __init__(self):
        self.logs = []
        
    def create_log(self,pop,gen,nevals,train_enemies,hof_gain,hof_defeat):
        log = {}
        train_gains = np.array([sum(ind.gains[train_enemies-1]) for ind in pop])
        all_gains = np.array([ind.sum_gain for ind in pop])
        all_defeated = np.array([ind.defeated for ind in pop])
        log['gen'] = gen
        log['nevals'] = nevals
        log['train_enemies_gain_mean'] = train_gains.mean()
        log['train_enemies_gain_max'] = train_gains.max()
        log['train_enemies_gain_std'] = train_gains.std()
        log['all_enemies_gain_mean'] = all_gains.mean()
        log['all_enemies_gain_max'] = all_gains.max()
        log['all_enemies_defeated_max'] = all_defeated.max()
        log['hof_gain_max'] = hof_gain['max_value']
        log['hof_defeat_max'] = hof_defeat['max_value']
        self.logs.append(log)
        self.pd = pd.DataFrame(self.logs)
        
def play(x):
    global env
    if x.evaluated:
        return x
    else:
        for enemy in range(1,9):
            env.update_parameter("enemies",[enemy])
            res = env.play(x.genes)
            x.fitness[enemy-1] = res[0]
            x.player_energy[enemy-1] = res[1]
            x.enemy_energy[enemy-1] = res[2]
        x.gains = (x.player_energy-x.enemy_energy)
        x.sum_gain = x.gains.sum()
        x.defeated = (x.enemy_energy==0).sum()
        x.evaluated = True
        return x

In [4]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5

In [5]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    child1.genes = (alpha*parent1.genes)+((1-alpha)*parent2.genes)
    child2.genes = (alpha*parent2.genes)+((1-alpha)*parent1.genes)
    return child1,child2

def cxBlend(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1.genes,))) - alpha
    child1.genes = (gamma*parent1.genes)+((1-gamma)*parent2.genes)
    child2.genes = (gamma*parent2.genes)+((1-gamma)*parent1.genes)
    return child1, child2

In [6]:
def mutGaussian(ind,sigma,indpb):
    new_ind = individual(0,len(ind.genes))
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind.genes))]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_ind = individual(0,len(ind.genes))
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind.genes),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def selTournament(individuals,k,tourn_size,train_enemies):
    selected = []
    for i in range(k):
        tourn_selection = np.random.choice(individuals,tourn_size)
        best = np.argmax([ind.gains[train_enemies-1].sum() for ind in tourn_selection])
        selected.append(tourn_selection[best])
    return selected

In [7]:
def genetic_algorithm(train_enemies,initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env

    train_enemies = np.array(train_enemies)
    pop = [individual(initial_range,individual_size) for i in range(pop_size)]
    hof_gain = {'best_solution':-np.inf,'max_value':-np.inf}
    hof_defeat = {'best_solution':-np.inf,'max_value':-np.inf}
    logbook = log_book()

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        pop = list(pool.map(play,pop))

        # record best solution
        for ind in pop:
            if ind.sum_gain>hof_gain['max_value']:
                hof_gain['best_solution'] = ind
                hof_gain['max_value'] = ind.sum_gain
            if ind.defeated>hof_defeat['max_value']:
                hof_defeat['best_solution'] = ind
                hof_defeat['max_value'] = ind.defeated
        
        logbook.create_log(pop,0,len(pop),train_enemies,hof_gain,hof_defeat)
        print("gen: 0")

        for g in range(ngen):
            # select parents
            parents = selTournament(individuals=pop,k=n_offspring*pop_size,tourn_size=tournsize_parent,train_enemies=train_enemies)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)        

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.evaluated]
            offspring = list(pool.map(play,offspring))

            # record best solution
            for ind in offspring:
                if ind.sum_gain>hof_gain['max_value']:
                    hof_gain['best_solution'] = ind
                    hof_gain['max_value'] = ind.sum_gain
                if ind.defeated>hof_defeat['max_value']:
                    hof_defeat['best_solution'] = ind
                    hof_defeat['max_value'] = ind.defeated

            # select the next generation of the population
            pop = selTournament(individuals=offspring,k=pop_size,tourn_size=tournsize_survival,train_enemies=train_enemies)

            logbook.create_log(pop,g+1,len(invalid_ind),train_enemies,hof_gain,hof_defeat)
            print("gen: "+str(g+1))
        pool.close()
        pool.join()

    display(logbook.pd)
    return hof_gain,hof_defeat,logbook
    

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]]),
              'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=30),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning_gain,hof_tuning_defeat
    iteration_logs = []
    for i in range(3):
        hof_gain,hof_defeat,logs = genetic_algorithm(**params)
        logs.pd['iteration'] = i+1
        iteration_logs.append(logs.pd)
        hof_tuning_gain.append(hof_gain)
        hof_tuning_defeat.append(hof_defeat)
    
    return pd.concat(iteration_logs).groupby('iteration')['all_enemies_gain_max'].agg(lambda x: np.mean(np.unique(x)[-5:])).mean()


In [10]:
hof_tuning_gain,hof_tuning_defeat = [], []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 00:06:58,944] A new study created in memory with name: GA
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 6, 7] which is of type list.
  warnings.warn(message)


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-213.692,64.6,78.591737,-658.836,-165.4,2,-165.4,2
1,1,231,-60.556,64.6,71.336087,-377.300,-165.4,3,-141.8,3
2,2,229,25.556,64.6,69.461002,-275.088,-165.4,3,-141.8,3
3,3,232,60.216,68.2,19.386669,-224.548,-165.4,3,-138.4,3
4,4,235,61.684,84.6,28.536632,-232.132,-132.2,5,-87.0,5
5,5,238,67.132,84.6,23.330516,-233.708,-100.2,3,-87.0,5
6,6,228,77.964,124.6,25.410886,-209.576,-100.2,3,-87.0,5
7,7,240,85.848,124.6,23.146751,-186.176,-109.0,3,-87.0,5
8,8,226,99.500,124.6,25.821239,-173.692,-101.0,3,-87.0,5
9,9,235,100.168,124.6,32.132310,-163.452,-95.4,3,-82.2,5


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-219.976,-22.4,62.432461,-677.076,-408.0,2,-408.0,2
1,1,235,-77.156,76.2,68.526343,-427.928,-210.0,4,-185.2,4
2,2,234,-6.200,84.0,75.608560,-306.892,-109.8,3,-109.8,4
3,3,226,69.784,107.6,29.265238,-194.828,-66.8,4,-66.8,4
4,4,233,81.448,107.4,13.497944,-151.880,-36.0,4,-36.0,4
5,5,233,94.784,127.0,15.205149,-150.360,-5.0,4,-5.0,4
6,6,229,88.720,127.0,27.694418,-151.264,-5.0,4,-5.0,4
7,7,237,92.188,127.0,40.651581,-107.280,12.8,4,12.8,4
8,8,236,101.052,127.0,43.396494,-84.624,-5.0,3,12.8,4
9,9,226,105.260,127.0,32.302254,-77.132,-5.0,3,12.8,4


...

gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-209.452,64.6,90.195998,-659.480,-225.4,2,-225.4,2
1,1,200,18.076,64.6,69.540479,-283.732,-140.2,3,-105.6,3
2,2,200,71.720,124.6,18.304535,-198.760,-139.4,3,-105.6,3
3,3,200,70.396,94.6,25.265771,-179.996,-92.2,3,-92.2,4
4,4,200,69.520,102.2,24.354252,-178.760,-78.8,3,-63.2,4
5,5,200,77.612,124.6,17.272378,-199.588,-99.0,3,-63.2,4
6,6,200,89.208,124.6,15.735359,-184.436,-89.6,3,-63.2,4
7,7,200,102.640,141.6,22.162996,-163.116,-99.0,3,-63.2,4
8,8,200,92.084,135.6,23.750405,-184.600,-121.0,3,-63.2,4
9,9,200,89.984,118.4,26.253315,-189.536,-125.4,3,-63.2,4


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5


Process ForkPoolWorker-270:
Process ForkPoolWorker-269:
Process ForkPoolWorker-271:
Process ForkPoolWorker-272:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    sel

KeyboardInterrupt: 

In [ ]:
#Trial 16 finished with value: 33.61333333333402 and parameters: {'train_enemies': [2, 5, 6], 'initial_range': 1, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 8, 'cxpb': 0.5, 'cx_type': 'whole_aritmetic', 'alpha': 0.2, 'mutpb': 1.0, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.6, 'tau': 0.30000000000000004, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}

In [22]:
params = {'train_enemies': [2, 5, 6],
          'initial_range': 1,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 8,
          'cxpb': 0.5,
          'cx_type': 'whole_aritmetic',
          'alpha': 0.2,
          'mutpb': 1.0,
          'mut_type': 'normal',
          'sigma': 2.5,
          'indpb': 0.6,
          'tau': 0.3,
          'tournsize_survival': 10,
          'ngen': 50,
          'verbose': 1}

In [ ]:
result_list = []
hof_list = []
for i in range(10):
    hof_gain,hof_defeat,logs = genetic_algorithm(**params)
    logs.pd['iteration'] = i+1
    result_list.append(logs.pd)
    hof_list.append(hof_gain)

gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30
gen: 31
gen: 32
gen: 33
gen: 34
gen: 35
gen: 36
gen: 37
gen: 38
gen: 39
gen: 40
gen: 41
gen: 42
gen: 43
gen: 44
gen: 45
gen: 46
gen: 47
gen: 48


In [ ]:
"""
params = {'train_enemies':[7,8],
          'initial_range':1,
          'pop_size':50,
          'n_offspring':2,
          'tournsize_parent':5,
          'cxpb':0.7,
          'cx_type':'blend',
          'alpha':'random',
          'mutpb':0.3,
          'mut_type':'adaptive',
          'sigma':2,
          'indpb':0.8,
          'tau':0.3,
          'tournsize_survival':10,
          'ngen':10,
          'verbose':1}
          
""""